# 6. 랭그래프로 설계하는 RAG 파이프라인

- 여러개의 에이전트로 구성된 LLM 애플리케이션과 RAG 파이프라인에서 자주 사용되는 순환과 분기를 포함한 복잡한 워크플로우를 간편하게 구현할 수 있도록 다양한 기능을 제공합니다.
- LLM을 사용하여 생성된 답변이 충분한지 혹은 답변을 재생성할지 결정합니다.
- LLM을 사용하여 어떤 툴을 호출할지 결정합니다.


## 1. 랭그래프의 구성요소

- 노드 : 그래프에서 개별 객체를 나타내는 요소입니다.
- 에지 : 두 노드를 연결하는 요소로, 노드 간의 관계 또는 경로를 나타냅니다.
- 기본 동작 방식은 노드가 작업을 완료하면 하나 이상의 에지를 통해 다른 노드에게 메시지를 보내고, 메시지를 받은 노드는 자신의 기능을 실행한 후 다음 노드로 메시지를 다시 전달하는 과정을 반복하는 방식입니다.


### 1.1. 그래프

- 랭그래프의 그래프는 슈퍼스텝방식으로 동작합니다.
- 슈퍼스텝은 그래프 처리 과정의 한 단계로, 각 노드가 병렬로 동시에 작업을 수행하는 단위입니다.
- 병렬로 작업을 수행한다는 것은 한 노드가 일을 끝낸 후에 다른 노드가 일을 시작하는 것이 아니라, 여러 노드가 동시에 자신의 일을 하는 모습을 뜻합니다.
- 동시에 실행되는 노드는 동일한 슈퍼스텝에 속하며, 순차적으로 실행되는 노드는 별도의 슈퍼스텝에 속합니다.
- 노드는 하나 이상의 입력 에지에서 새로운 메시지(상태)를 수신할 때 활성화됩니다.
- 활성화된 노드는 자신의 기능을 실행하고 처리 결과를 다른 노드로 전달합니다.
- 각 슈퍼스텝이 끝날 때, 입력 메시지가 없는 노드는 자신을 비활성화로 표시하여 완료된 상태로 표시합니다.
- 모든 노드가 비활성화되고 더 이상 메시지가 전송 중이지 않은 상태가 되면 그래프 실행이 종료됩니다.


- 상태 그래프 : 일반적으로 사용하는 그래프 클래스입니다.
- 메시지 그래프 : 오직 메시지 목록만으로 이루어지는 특별한 유형의 그래프 클래스입니다. 대화형 시스템에서 주로 사용되고, 메시지 흐름을 단순화하여 관리하기에 적합합니다.


### 1.2. 상태

- 가장 먼저 해야할 일은 그래프의 상태를 정의하는 것입니다.
- 상태란 애플리케이션 내에서 메시지로 주고받는 변수들의 집합입니다.


In [ ]:
from typing import TypedDict

class State(TypedDict):
    count: int
    messages: list[str]

- 아래는 상태가 변경되는 과정입니다.


In [1]:
import mermaid as md
render = md.Mermaid("""
%%{init: {'theme':'dark'}}%%
stateDiagram-v2
    n1 : 노드1<br><br>count=None<br>messages=[]
    n2 : 노드2<br><br>count=1<br>messages=["hi"]
    n3 : 노드3<br><br>count=2<br>messages=["hi"]
    n4 : 노드4<br><br>count=2<br>messages=["bye"]
    n1 --> n2 : count = 1<br>messages=["hi"]
    n2 --> n3 : count = 2
    n3 --> n4 : messages=["bye"]
""")
render

- 리듀서를 사용한다면 기존 상태에 새로운 업데이트를 결합하여 새로운 상태를 생성하는 것도 가능합니다.
- Annotated타입으로 리듀서 함수를 정의한다면 messages 변수는 리듀서 함수를 통해 업데이트 됩니다.
- 아래 예시에서는 messages에 add 리듀서가 지정되어 있어, 새로운 메시지가 추가될 때 기존 리스트와 병합됩니다.


In [ ]:
from typing import TypedDict, Annotated
from operator import add

class State(TypedDict):
    count: int
    messages: Annotated[list[str], add]

In [1]:
import mermaid as md
render = md.Mermaid("""
%%{init: {'theme':'dark'}}%%
stateDiagram-v2
    n1 : 노드1<br><br>count=None<br>messages=[]
    n2 : 노드2<br><br>count=1<br>messages=["hi"]
    n3 : 노드3<br><br>count=2<br>messages=["hi"]
    n4 : 노드4<br><br>count=2<br>messages=["hi","bye"]
    n1 --> n2 : count = 1<br>messages=["hi"]
    n2 --> n3 : count = 2
    n3 --> n4 : messages=["bye"]
""")
render

### 1.3 노드

- 노드는 실제 작업을 수행하는 실행 단위입니다.
- 에이전트의 로직을 담은 파이썬 함수가 곧 노드가 되며,각 노드는 상태를 입력을 받아 그 결과로 상태값을 업데이트하여 반환합니다.
- 노드는 정상적으로 실행될 수도 있고, 실패할 수도 있습니다.
- 노드는 첫번째 인자값으로 상태값을 받으며, 두번째 인자로 설정값을 받습니다.
- 이렇게 생성한 노드는 add_node() 메서드를 통해 그래프에 추가할 수 있습니다.


In [8]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph

# 상태 그래프 선언
builder = StateGraph(dict)

# 노드로 사용할 함수 정의
def my_node(state: dict, config: RunnableConfig):
    print("In node: ", config["configurable"]["user_id"])
    return {"results": f"Hello, {state['input']}!"}

def my_other_node(state: dict):
    return state # 상태를 그대로 반환

#노드를 그래프에 추가
builder.add_node("my_node", my_node)
builder.add_node("other_node", my_other_node)

- START 노드
  - 그래프 실행의 시작점을 나타내는 특별한 노드입니다.
  - 사용자 입력을 처음 받아 그래프로 전달하며, 그래프 내에서 첫번째로 실행될 노드를 지정할 때 사용합니다.


In [9]:
from langgraph.graph import START

builder.add_edge(START, "my_node")

- END 노드
  - 그래프 실행이 완료되었음을 나타내는 종료 노드입니다.
  - 특정 노드의 작업이 끝난 후 더 이상 처리할 작업이 없을 경우 END노드로 연결하여 그래프 실행을 종료합니다.


In [10]:
from langgraph.graph import END

builder.add_edge("other_node", END)

### 1.4 에지

- 그래프 내에서 노드가 작업을 수행한 후, 다음 동작을 결정하는 흐름 제어 요소입니다.
- 조건에 따라 분기하거나 종료를 지시할 수도 있습니다.


In [11]:
# 일반 에지
builder.add_edge("my_node", "other_node")

# 조건부 에지
# 특정 조건에 따라 다른 노드로 분기하거나 종료할 때 사용합니다.
def routing_function(state: dict):
    # 예시: count가 1이면 node_b로, 아니면 node_c로 이동
    return state.get("count", 0) == 1

builder.add_conditional_edges("my_node", routing_function, {True: "node_b", False: "node_c"})

#진입지점
#그래프가 시작될 때 처음 실행할 노드를 명시합니다. 주로 START 노드로 설정합니다.
builder.add_edge(START, "my_node")

# 조건부 진입지점
# 사용자의 입력이나 외부 조건에 따라 그래프 실행의 첫번째 노드를 동적으로 결정할 수 있습니다.
# 진입 지점 역시 조건부로 설정하여 다양한 초기 상태에 대응합니다.
builder.add_conditional_edges(START, routing_function, {True: "node_b", False: "node_c"})